# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
len(twits['data'])

1548010

### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'https?://\S+', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'$', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'@\S+', ' ', text)

    # Replace everything not a letter with a space
    #text = re.sub(r'^[\s]', '', text)
    #text = re.sub(r'.\s', '', text)
    #text = re.sub(r'\W', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    #tokens = re.split('\s+', text)
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(t) for t in tokens]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
preprocess('RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG')

['rt',
 'our',
 'annual',
 'look',
 'at',
 'the',
 'year',
 'in',
 'google',
 'blogging',
 'and',
 'beyond',
 'goog']

In [7]:
messages

['$FITB great buy at 26.00...ill wait',
 '@StockTwits $MSFT',
 '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating',
 '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.',
 '$AMD reveal yourself!',
 '$AAPL Why the drop? I warren Buffet taking out his position?',
 '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA',
 '$BAC ok good we&#39;re not dropping in price over the weekend, lol',
 '$AMAT - Daily Chart, we need to get back to above 50.',
 '$GME 3% drop per week after spike... if no news in 3 months, back to 12s... if BO, then bingo... what is the odds?',
 '$SBUX STRONG BUY!',
 '$SNPS short ratio is 2.17 at 2018-06-15 and short % to float is 1.42% http://sunshineavenue.com/stock/SNPS/ via @sunshineave',
 '$NFLX price squeezing,perfect place for an option straddle near the supporting

### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [8]:
# TODO Implement

tokenized = [preprocess(messages_list) for messages_list in messages]

In [9]:
tokenized

[['fitb', 'great', 'buy', 'at', '26', '00', 'ill', 'wait'],
 ['msft'],
 ['staanalystalert',
  'for',
  'tdg',
  'jefferies',
  'maintains',
  'with',
  'a',
  'rating',
  'of',
  'hold',
  'setting',
  'target',
  'price',
  'at',
  'usd',
  '350',
  '00',
  'our',
  'own',
  'verdict',
  'is',
  'buy'],
 ['amd',
  'i',
  'heard',
  'there',
  's',
  'a',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'on',
  'stocktwits'],
 ['amd', 'reveal', 'yourself'],
 ['aapl',
  'why',
  'the',
  'drop',
  'i',
  'warren',
  'buffet',
  'taking',
  'out',
  'his',
  'position'],
 ['ba',
  'bear',
  'have',
  '1',
  'reason',
  'on',
  '06',
  '29',
  'to',
  'pay',
  'more',
  'attention'],
 ['bac',
  'ok',
  'good',
  'we',
  '39',
  're',
  'not',
  'dropping',
  'in',
  'price',
  'over',
  'the',
  'weekend',
  'lol'],
 ['amat',
  'daily',
  'chart',
  'we',
  'need',
  'to',
  'get',
  'back',
  'to',
  'above',
  '50'],
 ['gme',
  '3',
 

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [10]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 


tokenized_list = []

for i in tokenized:
    for j in i:
        tokenized_list.append(j)


bow = Counter(tokenized_list)



In [11]:
bow

Counter({'fitb': 693,
         'great': 17571,
         'buy': 76083,
         'at': 138334,
         '26': 15930,
         '00': 23623,
         'ill': 881,
         'wait': 11350,
         'msft': 32485,
         'staanalystalert': 4072,
         'for': 273519,
         'tdg': 504,
         'jefferies': 1057,
         'maintains': 903,
         'with': 68854,
         'a': 326379,
         'rating': 15436,
         'of': 211302,
         'hold': 22135,
         'setting': 3889,
         'target': 16568,
         'price': 37078,
         'usd': 2922,
         '350': 2570,
         'our': 12205,
         'own': 8967,
         'verdict': 3736,
         'is': 284869,
         'amd': 264359,
         'i': 183058,
         'heard': 1357,
         'there': 25944,
         's': 194607,
         'guy': 13422,
         'who': 17885,
         'know': 19932,
         'someone': 5192,
         'think': 24281,
         'somebody': 1100,
         'something': 6009,
         'on': 241882,
         '

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [12]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 

# Dictionart that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
total_count = len(tokenized_list)
freqs = {word: count/total_count for word, count in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 0.0000002

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 10

# The k most common words in the corpus. Use `high_cutoff` as the k.

K_most_common = dict(bow.most_common(high_cutoff))

filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 


{'the': 398775, 'to': 379409, 'a': 326379, 'is': 284869, 'for': 273519, 'amd': 264359, 'on': 241882, 'of': 211302, 'and': 208517, 'in': 205266}


37426

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [13]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement

# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab = {word: num_id for word, num_id in enumerate(filtered_words)}
#print(vocab.items())

# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {num_id : word for (num_id, word) in vocab.items()}
print(id2vocab)



{0: 'fitb', 1: 'great', 2: 'buy', 3: 'at', 4: '26', 5: '00', 6: 'ill', 7: 'wait', 8: 'msft', 9: 'staanalystalert', 10: 'tdg', 11: 'jefferies', 12: 'maintains', 13: 'with', 14: 'rating', 15: 'hold', 16: 'setting', 17: 'target', 18: 'price', 19: 'usd', 20: '350', 21: 'our', 22: 'own', 23: 'verdict', 24: 'i', 25: 'heard', 26: 'there', 27: 's', 28: 'guy', 29: 'who', 30: 'know', 31: 'someone', 32: 'think', 33: 'somebody', 34: 'something', 35: 'stocktwits', 36: 'reveal', 37: 'yourself', 38: 'aapl', 39: 'why', 40: 'drop', 41: 'warren', 42: 'buffet', 43: 'taking', 44: 'out', 45: 'his', 46: 'position', 47: 'ba', 48: 'bear', 49: 'have', 50: '1', 51: 'reason', 52: '06', 53: '29', 54: 'pay', 55: 'more', 56: 'attention', 57: 'bac', 58: 'ok', 59: 'good', 60: 'we', 61: '39', 62: 're', 63: 'not', 64: 'dropping', 65: 'over', 66: 'weekend', 67: 'lol', 68: 'amat', 69: 'daily', 70: 'chart', 71: 'need', 72: 'get', 73: 'back', 74: 'above', 75: '50', 76: 'gme', 77: '3', 78: 'per', 79: 'week', 80: 'after', 81

In [14]:
# tokenized with the words not in `filtered_words` removed.
#filtered = [word for word in tokenized_list if word not in filtered_words]

from tqdm import tqdm as tqdm

counts = Counter(filtered_words)

i=1
vocab={}

for w in (filtered_words):
    vocab[w]=i
    i+=1
    
id2vocab = { ii:word for  word,ii in vocab.items()}
filtered=[[word for word in message if word in vocab] for message in tqdm(tokenized)]

100%|██████████| 1548010/1548010 [00:10<00:00, 141859.85it/s]


### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [15]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral

for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]
    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment) 

If you did it correctly, you should see the following result 

In [16]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.1990822691048325

Finally let's convert our tokens into integer ids which we can pass to the network.

In [17]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [18]:
'''
# PL added
# First checking if GPU is available
train_on_gpu=torch.cuda.is_available()

if(train_on_gpu):
    print('Training on GPU.')
else:
    print('No GPU available, training on CPU.')
'''

"\n# PL added\n# First checking if GPU is available\ntrain_on_gpu=torch.cuda.is_available()\n\nif(train_on_gpu):\n    print('Training on GPU.')\nelse:\n    print('No GPU available, training on CPU.')\n"

In [19]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        """
        Initialize the model by setting up the layers.
        
        Parameters
        ----------
            vocab_size : The vocabulary size.
            embed_size : The embedding layer size.
            lstm_size : The LSTM layer size.
            output_size : The output size.
            lstm_layers : The number of LSTM layers.
            dropout : The dropout probability.
        """
        
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.output_size = output_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout
        
        # TODO Implement

        # Setup embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, 
                            dropout=dropout, batch_first=False)
        
        # Setup additional layers
        self.fc = nn.Linear(lstm_size, output_size)
        self.sm = nn.LogSoftmax(dim=1)
        self.dropout = nn.Dropout(dropout)


    def init_hidden(self, batch_size):
        """ 
        Initializes hidden state
        
        Parameters
        ----------
            batch_size : The size of batches.
        
        Returns
        -------
            hidden_state
            
        """
        
        # TODO Implement 
        
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        
        weight = next(self.parameters()).data
        
        hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        
        '''
        
        if (train_on_gpu):
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_().cuda())
        else:
            hidden = (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                      weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())
        '''
        return hidden


    def forward(self, nn_input, hidden_state):
        """
        Perform a forward pass of our model on nn_input.
        
        Parameters
        ----------
            nn_input : The batch of input to the NN.
            hidden_state : The LSTM hidden state.

        Returns
        -------
            logps: log softmax output
            hidden_state: The new hidden state.

        """
        
        # TODO Implement 
        
        batch_size = nn_input.size(1)

        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden_new = self.lstm(embeds, hidden_state)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.lstm_size)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # softmax function
        logps = self.sm(out)

        # reshape to be batch_size last
        logps = logps.view(-1, batch_size, self.output_size)
        logps = logps[-1, :, :] 

        
        return logps, hidden_new

### View Model

In [20]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.4540, -1.2995, -1.8231, -1.8102, -1.7805],
        [-1.4473, -1.3012, -1.8330, -1.8114, -1.7765],
        [-1.4567, -1.2964, -1.8289, -1.8014, -1.7849],
        [-1.4496, -1.2989, -1.8309, -1.8045, -1.7856]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [21]:
def dataloader(messages, labels, sequence_length=30, batch_size=32, shuffle=False):
    """ 
    Build a dataloader.
    """
    if shuffle:
        indices = list(range(len(messages)))
        random.shuffle(indices)
        messages = [messages[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(messages)

    for ii in range(0, total_sequences, batch_size):
        batch_messages = messages[ii: ii+batch_size]
        
        # First initialize a tensor of all zeros
        batch = torch.zeros((sequence_length, len(batch_messages)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_messages):
            token_tensor = torch.tensor(tokens)
            # Left pad!
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_messages)])
        
        yield batch, label_tensor

In [22]:
len(token_ids)

1053032

In [23]:
len(sentiments)

1053032

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [24]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   

# TODO Implement 

from sklearn.model_selection import train_test_split

train_features, valid_features, train_labels, valid_labels = train_test_split(token_ids, sentiments, test_size=0.2, random_state=42)

'''
train_features = pass
valid_features = pass
train_labels = pass
valid_labels = pass
'''

'\ntrain_features = pass\nvalid_features = pass\ntrain_labels = pass\nvalid_labels = pass\n'

In [25]:
len(train_features)

842425

In [26]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(37427, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
  (sm): LogSoftmax()
  (dropout): Dropout(p=0.2)
)

In [28]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""
import numpy as np 

epochs = 2
batch_size = 1024
learning_rate = 0.001
clip=5 # gradient clipping

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(
            train_features, train_labels, batch_size=batch_size, sequence_length=20, shuffle=True):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        hidden = tuple([each.data for each in hidden])
        
        model.zero_grad()
        
        log_ps, hidden = model(text_batch, hidden)
        
        loss = criterion(log_ps, labels)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        if steps % print_every == 0:

            
            model.eval()
            
            # TODO Implement: Print metrics
            val_losses = []
            accuracy = []
            
            for text_batch, labels in dataloader(valid_features, valid_labels, 
                                                 batch_size = batch_size, sequence_length=20, shuffle=True):
                
                # Set Device
                #print('labels.shape[0]: ', labels.shape[0])
                val_hidden = model.init_hidden(labels.shape[0])
                text_batch, labels = text_batch.to(device), labels.to(device)
                for each in val_hidden:
                    each.to(device)
                
                val_hidden = tuple([each.data for each in val_hidden])
                
                val_log_ps, val_hidden = model(text_batch, val_hidden)
                val_loss = criterion(val_log_ps.squeeze(), labels)
                val_losses.append(val_loss.item())
                ps = torch.exp(val_log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                
                equals = top_class == labels.view(*top_class.shape)
            accuracy.append(torch.mean(equals.type(torch.FloatTensor)).item())

            
            model.train()
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)),
                  "Accuracy: {:.6f}".format(np.mean(accuracy)))

Starting epoch 1
Epoch: 1/2... Step: 100... Loss: 0.904701... Val Loss: 0.946333 Accuracy: 0.666667
Epoch: 1/2... Step: 200... Loss: 0.814881... Val Loss: 0.838546 Accuracy: 0.650655
Epoch: 1/2... Step: 300... Loss: 0.743627... Val Loss: 0.784776 Accuracy: 0.698690
Epoch: 1/2... Step: 400... Loss: 0.784941... Val Loss: 0.749332 Accuracy: 0.724891
Epoch: 1/2... Step: 500... Loss: 0.734792... Val Loss: 0.728584 Accuracy: 0.703057
Epoch: 1/2... Step: 600... Loss: 0.742165... Val Loss: 0.707232 Accuracy: 0.724891
Epoch: 1/2... Step: 700... Loss: 0.685432... Val Loss: 0.695452 Accuracy: 0.723435
Epoch: 1/2... Step: 800... Loss: 0.703390... Val Loss: 0.685087 Accuracy: 0.735080
Starting epoch 2
Epoch: 2/2... Step: 100... Loss: 0.630048... Val Loss: 0.679476 Accuracy: 0.703057
Epoch: 2/2... Step: 200... Loss: 0.650738... Val Loss: 0.677786 Accuracy: 0.726346
Epoch: 2/2... Step: 300... Loss: 0.609162... Val Loss: 0.669265 Accuracy: 0.742358
Epoch: 2/2... Step: 400... Loss: 0.648682... Val Loss

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [64]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    # Filter non-vocab words
    tokens = [word for word in tokens if word in vocab]
    
    # Convert words to ids
    tokens = [vocab[i] for i in tokens]
        
    # Adding a batch dimension
    text_input = torch.from_numpy(np.asarray(torch.FloatTensor(tokens).view(-1, 1)))
    # Get the NN output
    hidden = model.init_hidden(1)
    logps, _ = model.forward(text_input, hidden)
    # Take the exponent of the NN output to get a range of 0 to 1 for each label.

    pred = np.exp((logps[0]).detach().numpy())
    
    return pred

In [65]:
text = "Google is working on self driving cars, I'm bullish on $goog"
model.eval()
model.to("cpu")
predict(text, model, vocab)

array([  1.33163863e-04,   1.26937162e-02,   8.58188700e-03,
         8.78889561e-01,   9.97017026e-02], dtype=float32)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

(1) What is the prediction of the model?
Answer: The model provides predicted values for 5 possible sentiments. The highest predicted value is potential prediction.

(2) What is the uncertainty of the prediction?
Answer: The prediction accuracy relies on number of batches, epoches, training data for training. In this case, increasing number of epoches could improve the prediction accuracy. Moroever, if we predict the data (text) which the model haven't seen before OR is not related to the training data, the model might fail to predict. 

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [66]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [67]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [68]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [69]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': array([ 0.22142003,  0.10094947,  0.33222291,  0.27886292,  0.06654464], dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.